# CuPyとNumPyの比較

In [14]:
# -*- coding: utf-8 -*-
import numpy as np
import cupy as cp
import gc

### 備考
CuPyのスクリプトで，次の２行を入れている理由<br>
  del a, b, c<br>
  gc.collect()<br>
%%timeit　が繰返し計算を実施するため，配列のメモリ確保が増えて，CuPyの場合に計算時間が大幅に変動する（標準偏差が大きい）現象が見られた。<br>
推測として，GPUは使用したメモリの解法が上手でない，または，メモリの断片化が生じていると考えた。<br>
そこで，明示的に del:メモリの解法，gc.collect():ガベージコレクション，を実施した。<br>
その結果，計算時間の標準偏差が小さくなった。<br>
このトレードオフとして，計算時間が多少長くなり，純粋にNumPyとの比較が難しくなった。<br>
この対策を行わないときは，ほんの数回または１回だけの試行とする。
実際のコーディングで，大きなメモリを確保して，この問題を避けるには，
CuPyのメモリマネジメント https://docs-cupy.chainer.org/en/stable/reference/memory.html<br>
を読まれたい。この中に，MemoryPoolという方法がある。<br>
https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.cuda.MemoryPool.html<br>
または，Low-Level CUDA Support https://docs-cupy.chainer.org/en/stable/reference/cuda.html<br>
があり，この中に，cupy.cuda.set_allocator<br>
https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.cuda.set_allocator.html?highlight=set_allocator<br>
があり，各自で調べられたい。


# 線形代数の計算 ( Calculation of Linear Algebra )
Numpy Linear algebra (numpy.linalg): https://docs.scipy.org/doc/numpy/reference/routines.linalg.html<br>
Cupy  Linear algebra (cupy.linalg): https://docs-cupy.chainer.org/en/stable/reference/linalg.html

### 内積（inner product）
numpy.inner: https://docs.scipy.org/doc/numpy/reference/generated/numpy.inner.html<br>
cupy.inner: https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.inner.html

In [15]:
N = int(10**3)

In [16]:
%%timeit -r3 -n10
a_np = np.random.rand(N)
b_np = np.random.rand(N)
c_np = np.inner(a_np, b_np)

24.6 µs ± 3.59 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [17]:
%%timeit -r3 -n10
a_cp = cp.random.rand(N)
b_cp = cp.random.rand(N)
c_cp = cp.inner(a_cp, b_cp)

#gc.collect()

95.2 µs ± 17.2 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


### 行列の乗算（matrix multiplication）
注意：乗算にdot関数を用いているが，乗算＝dot積ではなく，dot積（Dot product）の一部に乗算演算があると考える。そのため，numpy, cupy共に，引数を変えると，通常の行列乗算以外のdot積を計算できる<br>
numpy.dot: https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html<br>
cupy.dot: https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.dot.html

In [18]:
N = int(10**3)

In [19]:
%%timeit -r10 -n20
A_np = np.random.rand(N,N)
B_np = np.random.rand(N,N)
C_np = np.dot(A_np, B_np)

42.5 ms ± 3.06 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


次のCuPyの試行は，試行を経るほど，処理時間が大きく異なることがある。

In [20]:
%%timeit -r10 -n20
A_cp = cp.random.rand(N,N)
B_cp = cp.random.rand(N,N)
C_cp = cp.dot(A_cp, B_cp)
#del a_cp, b_cp, c_cp
#gc.collect()

90.6 µs ± 28.5 µs per loop (mean ± std. dev. of 10 runs, 20 loops each)


### ノルム(norm)
numpy.linalg.norm: https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html<br>
cupy: https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.linalg.norm.html

$$
  ベクトルa(m)と行列B(n,m)を用い，a(m)とB(i,m), (i=1-n)のノルムをn個求める
$$
np.linalg.norm()のパラメータaxisについて，axis=1は行列の行ごとのベクトルをとる。ちなみに，axis=0は列毎のベクトルノルムをとる。<br>
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html

In [21]:
N = int(10**2)

In [22]:
%%timeit -r2 -n3
a_np = np.random.rand(N)
B_np = np.random.rand(N**3, N)  
np.linalg.norm(a_np - B_np, axis=1)

1.69 s ± 28.1 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [23]:
%%timeit -r2 -n3
a_cp = cp.random.rand(N)
B_cp = cp.random.rand(N**3, N)
cp.linalg.norm(a_cp - B_cp, axis=1)

#del a_cp, B_cp  # Delete memories
#gc.collect()    # Garbage collection

333 µs ± 179 µs per loop (mean ± std. dev. of 2 runs, 3 loops each)


# 初等関数の計算 ( Calculation of Elementary Function )
多項式，sin, cos, expを含む計算の経過時間を比較する

### 円周率の計算，モンテカルロ法を用いる
２編，multiprocessingを参照<，この計算は，2乗計算の多項式と代数計算を含む

In [24]:
N = int(10**7)

In [25]:
%%timeit -r2 -n5
x_np = np.random.uniform(0,1,N)
y_np = np.random.uniform(0,1,N)
r_np = np.add(x_np**2, y_np**2)
unit_np = np.ones(N)
count_np = np.less_equal(r_np, unit_np)
pi = 4.0*count_np.sum()/np.float(N)

351 ms ± 4.45 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [26]:
%%timeit -r2 -n5
x_cp = cp.random.uniform(0,1,N)
y_cp = cp.random.uniform(0,1,N)
r_cp = cp.add(x_cp**2, y_cp**2)
unit_cp = cp.ones(N)
count_cp = cp.less_equal(r_cp, unit_cp)
pi = 4.0*count_cp.sum()/cp.float(N)

The slowest run took 1701.48 times longer than the fastest. This could mean that an intermediate result is being cached.
172 ms ± 172 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


### シグモイド関数の計算，expの計算を含む

In [27]:
def numpy_sigmoid(x):
    return 1/(1+np.exp(-x))

def cupy_sigmoid(x):
    return 1/(1+cp.exp(-x))

In [28]:
N = int(10**7)

In [29]:
%%timeit -r3 -n10
x_np = np.random.rand(N)
numpy_sigmoid(x_np)

239 ms ± 3.23 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [30]:
%%timeit -r3 -n10
x_cp = cp.random.rand(N)
cupy_sigmoid(x_cp)

The slowest run took 1005.61 times longer than the fastest. This could mean that an intermediate result is being cached.
29.8 ms ± 42.1 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


### FFT（fast fourier transform）の計算
sin, cosの計算と配列操作の処理を含む

In [31]:
N = int(2**17)

In [32]:
%%timeit -r3 -n7
x_np = np.random.rand(N)
np.fft.fft(x_np)

3.66 ms ± 1.46 ms per loop (mean ± std. dev. of 3 runs, 7 loops each)


In [33]:
%%timeit -r3 -n7
x_cp = cp.random.rand(N)
cp.fft.fft(x_cp)

The slowest run took 303.33 times longer than the fastest. This could mean that an intermediate result is being cached.
33.1 ms ± 46.4 ms per loop (mean ± std. dev. of 3 runs, 7 loops each)
